## Importing necessary libraries

In [1]:
# importing inltk and its dependencies
import torch
from inltk.inltk import tokenize
from inltk.inltk import get_embedding_vectors

# importing for data exploration and analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# classical model - random forest
from sklearn.ensemble import RandomForestClassifier

# deep learning model - cnn, lstm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D,LSTM

# importing for training and test split
from sklearn.model_selection import train_test_split

# Tensorboard for visualisation
from tensorflow.keras.callbacks import TensorBoard
from time import time

## Reading Data

In [2]:
data = pd.read_csv("q1.csv")

## Preprocessing Data

In [3]:
%%time

# converting marks into int to prevent errors during training (as classification task requires int classes)
data = data.astype({'marks' : int})

# removing the nan ans from the dataframe
data = data[data['ans'].notna()]

# after droping the index gets reorganised, resetting it here
data.reset_index(drop= True, inplace= True)

CPU times: user 6.79 ms, sys: 0 ns, total: 6.79 ms
Wall time: 6.68 ms


In [4]:
%%time

# Feature extraction - No. of words
# creating a new series
no_of_words = pd.Series([], dtype = int) 

for row_index,row in data.iterrows():
    no = len(tokenize(row['ans'], 'or'))
    no_of_words[row_index] = no
    
# adding the created series into the dataframe at position 2
data.insert(2, "no_of_words", no_of_words)

pad_length = 124  # median no of words

CPU times: user 6.56 s, sys: 65 ms, total: 6.62 s
Wall time: 6.62 s


In [5]:
%%time

# Vectorising Answer Text
# creating a new series
vectors = pd.Series([], dtype = float) 
    
for row_index,row in data.iterrows():
    vect = get_embedding_vectors(row['ans'], 'or')     # getting embedding vectors
    vectors[row_index] =  vect
    
# Padding and trimming the vectors to a common length
pad_vectors = keras.preprocessing.sequence.pad_sequences(vectors, padding="post", maxlen=pad_length, dtype='float32')

/home/deepak/Documents/answer_evaluator/venv/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/deepak/Documents/answer_evaluator/venv/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'fastai.text.models.awd_lstm.AWD_LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/deepak/Documents/answer_evaluator/venv/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embe

CPU times: user 13.4 s, sys: 3.24 s, total: 16.6 s
Wall time: 16.9 s


In [6]:
# checking if padding done correctly
for _ in pad_vectors:
    if len(_) != pad_length:
        print(pad_vectors.index(_))

## Training and Testing

### Function for classical model - Random Forest Classifier

In [7]:
def train_rfc(x_train, x_test, y_train, y_test, n):
    # Reshaping the vector input into 2-d from 3-d as RFC exxcepts <=2d input
    nsamples, nx, ny = x_train.shape
    x_train_rfc = x_train.reshape((nsamples,nx*ny))

    nsamples, nx, ny = x_test.shape
    x_test_rfc = x_test.reshape((nsamples,nx*ny))

    rfc = RandomForestClassifier(n_estimators=26, random_state=200)
    clf = rfc.fit(x_train_rfc, y_train)
    acc = clf.score(x_test_rfc, y_test)*100
    return acc

### Function for DL model - CNN

In [8]:
def train_cnn(x_train, x_test, y_train, y_test, n):
    #Add Sequential
    model_cnn = keras.Sequential()

    # 1st convolutional layer
    model_cnn.add(Dropout(0.2))
    model_cnn.add(Conv1D(64, 4, activation='relu'))
    model_cnn.add(MaxPooling1D(pool_size=8))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(60,activation='relu'))
    model_cnn.add(Dense(6,activation='softmax'))

    model_cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
#     Create a TensorBoard instance with the path to the logs directory
    tensorboard = TensorBoard(log_dir='logs/{}/cnn/{}'.format(n, time()))
    
    model_cnn.fit(x_train, y_train, epochs = 10,  verbose = 1, callbacks=[tensorboard])

    results = model_cnn.evaluate(x_test, y_test)
    return results[1]*100

### Function for DL model - LSTM

In [9]:
def train_lstm(x_train, x_test, y_train, y_test, n):
    model_lstm = keras.Sequential()

    model_lstm.add(Conv1D(input_shape=(pad_length,400), filters = pad_length, kernel_size = 40))
    model_lstm.add(LSTM(96, dropout=0.2))
    model_lstm.add(Dense(24,activation='sigmoid'))
    model_lstm.add(Dense(6,activation='softmax'))

    model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#     Create a TensorBoard instance with the path to the logs directory
    tensorboard = TensorBoard(log_dir='logs/{}/lstm/{}'.format(n, time()))

    fitmodel = model_lstm.fit(x_train, y_train, epochs = 10,  verbose = 1, callbacks=[tensorboard])   

    results = model_lstm.evaluate(x_test, y_test)
    return results[1]*100

### Training

In [10]:
%%time

acc_rfc_list = []
acc_cnn_list = []
acc_lstm_list = []
acc = {}

for n in [round(_ * 0.1, 1) for _ in range(2, 9)]:
    print("Training on:",n,"\n")
#     splitting data into training and testing set
    x_train, x_test, y_train, y_test = train_test_split(pad_vectors, data['marks'], test_size=round(1-n,1), random_state=22)
    
#     calling rfc for training
    rfc_acc = train_rfc(x_train, x_test, y_train, y_test, n)
    acc_rfc_list.append(rfc_acc)
    acc[n] = [rfc_acc]
    
#     converting input to tensors for input into DL models
    x_train_dl = tf.convert_to_tensor(x_train, np.float32)
    y_train_dl = tf.convert_to_tensor(y_train, np.float32)

    x_test_dl = tf.convert_to_tensor(x_test, np.float32)
    y_test_dl = tf.convert_to_tensor(y_test, np.float32)
    
    print("CNN Training--------------------------------------------",n,"\n")    
#     calling cnn for training
    cnn_acc = train_cnn(x_train_dl, x_test_dl, y_train_dl, y_test_dl, n)
    acc_cnn_list.append(cnn_acc)
    acc[n].append(cnn_acc)
    
    print("\nLSTM Training--------------------------------------------",n,"\n")
#     calling lstm for training
    lstm_acc = train_lstm(x_train_dl, x_test_dl, y_train_dl, y_test_dl, n)
    acc_lstm_list.append(lstm_acc)
    acc[n].append(lstm_acc)
    
    print("\n-------------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------------\n\n\n\n")

Training on: 0.2 

CNN Training-------------------------------------------- 0.2 

Train on 13 samples
Epoch 1/10
13/13 [==============================] - 1s 85ms/sample - loss: 2.1139 - accuracy: 0.3846
Epoch 2/10
13/13 [==============================] - 0s 4ms/sample - loss: 1.3043 - accuracy: 0.3077
Epoch 3/10
13/13 [==============================] - 0s 3ms/sample - loss: 0.6406 - accuracy: 0.8462
Epoch 4/10
13/13 [==============================] - 0s 3ms/sample - loss: 0.5163 - accuracy: 0.9231
Epoch 5/10
13/13 [==============================] - 0s 4ms/sample - loss: 0.3332 - accuracy: 0.9231
Epoch 6/10
13/13 [==============================] - 0s 4ms/sample - loss: 0.1192 - accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 0s 5ms/sample - loss: 0.0607 - accuracy: 1.0000
Epoch 8/10
13/13 [==============================] - 0s 4ms/sample - loss: 0.0532 - accuracy: 1.0000
Epoch 9/10
13/13 [==============================] - 0s 4ms/sample - loss: 0.0302 - accuracy: 1.00

20/20 [==============================] - 1s 68ms/sample - loss: 2.4726 - accuracy: 0.1500
Epoch 2/10
20/20 [==============================] - 0s 4ms/sample - loss: 1.2649 - accuracy: 0.5500
Epoch 3/10
20/20 [==============================] - 0s 3ms/sample - loss: 1.0452 - accuracy: 0.6500
Epoch 4/10
20/20 [==============================] - 0s 4ms/sample - loss: 0.6839 - accuracy: 0.8500
Epoch 5/10
20/20 [==============================] - 0s 4ms/sample - loss: 0.4748 - accuracy: 0.9000
Epoch 6/10
20/20 [==============================] - 0s 4ms/sample - loss: 0.3047 - accuracy: 0.9000
Epoch 7/10
20/20 [==============================] - 0s 3ms/sample - loss: 0.2014 - accuracy: 1.0000
Epoch 8/10
20/20 [==============================] - 0s 4ms/sample - loss: 0.0961 - accuracy: 1.0000
Epoch 9/10
20/20 [==============================] - 0s 3ms/sample - loss: 0.0585 - accuracy: 1.0000
Epoch 10/10
47/1 [============================================================================================


LSTM Training-------------------------------------------- 0.4 

Train on 26 samples
Epoch 1/10
26/26 [==============================] - 5s 203ms/sample - loss: 2.0427 - accuracy: 0.2308
Epoch 2/10
26/26 [==============================] - 2s 58ms/sample - loss: 1.6666 - accuracy: 0.2308
Epoch 3/10
26/26 [==============================] - 0s 18ms/sample - loss: 1.4864 - accuracy: 0.2308
Epoch 4/10
26/26 [==============================] - 0s 19ms/sample - loss: 1.3366 - accuracy: 0.3077
Epoch 5/10
26/26 [==============================] - 0s 19ms/sample - loss: 1.2578 - accuracy: 0.3846
Epoch 6/10
26/26 [==============================] - 0s 17ms/sample - loss: 1.1831 - accuracy: 0.4231
Epoch 7/10
26/26 [==============================] - 1s 23ms/sample - loss: 1.1147 - accuracy: 0.6923
Epoch 8/10
26/26 [==============================] - 0s 17ms/sample - loss: 1.0656 - accuracy: 0.7692
Epoch 9/10
26/26 [==============================] - 0s 17ms/sample - loss: 1.0220 - accuracy: 0.8077
Epoch

33/33 [==============================] - 1s 19ms/sample - loss: 1.3922 - accuracy: 0.5152
Epoch 8/10
33/33 [==============================] - 1s 18ms/sample - loss: 1.3403 - accuracy: 0.5455
Epoch 9/10
33/33 [==============================] - 1s 18ms/sample - loss: 1.2932 - accuracy: 0.5455
Epoch 10/10
34/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

46/46 [==============================] - 1s 16ms/sample - loss: 2.3568 - accuracy: 0.1957
Epoch 2/10
46/46 [==============================] - 0s 2ms/sample - loss: 1.5257 - accuracy: 0.3261
Epoch 3/10
46/46 [==============================] - 0s 2ms/sample - loss: 1.0648 - accuracy: 0.7391
Epoch 4/10
46/46 [==============================] - 0s 2ms/sample - loss: 0.8160 - accuracy: 0.7174
Epoch 5/10
46/46 [==============================] - 0s 2ms/sample - loss: 0.5729 - accuracy: 0.8261
Epoch 6/10
46/46 [==============================] - 0s 2ms/sample - loss: 0.3975 - accuracy: 0.9348
Epoch 7/10
46/46 [==============================] - 0s 2ms/sample - loss: 0.2911 - accuracy: 0.9783
Epoch 8/10
46/46 [==============================] - 0s 3ms/sample - loss: 0.2259 - accuracy: 1.0000
Epoch 9/10
46/46 [==============================] - 0s 2ms/sample - loss: 0.1647 - accuracy: 1.0000
Epoch 10/10
21/1 [============================================================================================

53/53 [==============================] - 4s 84ms/sample - loss: 1.7731 - accuracy: 0.2075
Epoch 2/10
53/53 [==============================] - 1s 17ms/sample - loss: 1.4763 - accuracy: 0.5660
Epoch 3/10
53/53 [==============================] - 1s 16ms/sample - loss: 1.3103 - accuracy: 0.6415
Epoch 4/10
53/53 [==============================] - 1s 17ms/sample - loss: 1.1883 - accuracy: 0.6415
Epoch 5/10
53/53 [==============================] - 1s 17ms/sample - loss: 1.0858 - accuracy: 0.6415
Epoch 6/10
53/53 [==============================] - 1s 17ms/sample - loss: 1.0086 - accuracy: 0.7170
Epoch 7/10
53/53 [==============================] - 1s 16ms/sample - loss: 0.9472 - accuracy: 0.7925
Epoch 8/10
53/53 [==============================] - 1s 17ms/sample - loss: 0.9016 - accuracy: 0.8302
Epoch 9/10
53/53 [==============================] - 1s 16ms/sample - loss: 0.8534 - accuracy: 0.8491
Epoch 10/10
14/1 [====================================================================================

In [11]:
print("RFC:",acc_rfc_list)
print("CNN:",acc_cnn_list)
print("LSTM:",acc_lstm_list)

RFC: [35.18518518518518, 44.680851063829785, 39.02439024390244, 52.94117647058824, 51.85185185185185, 57.14285714285714, 42.857142857142854]
CNN: [38.88888955116272, 48.93617033958435, 48.78048896789551, 50.0, 48.148149251937866, 57.14285969734192, 64.28571343421936]
LSTM: [44.44444477558136, 57.446807622909546, 51.21951103210449, 32.35294222831726, 48.148149251937866, 61.90476417541504, 71.42857313156128]


In [12]:
acc

{0.2: [35.18518518518518, 38.88888955116272, 44.44444477558136],
 0.3: [44.680851063829785, 48.93617033958435, 57.446807622909546],
 0.4: [39.02439024390244, 48.78048896789551, 51.21951103210449],
 0.5: [52.94117647058824, 50.0, 32.35294222831726],
 0.6: [51.85185185185185, 48.148149251937866, 48.148149251937866],
 0.7: [57.14285714285714, 57.14285969734192, 61.90476417541504],
 0.8: [42.857142857142854, 64.28571343421936, 71.42857313156128]}